In [ ]:
###IMPORT
import numpy as np
import pandas as pd
import dataprocessing
from stochasticmodel import StochasticLoadModel, StochasticSimulationResult
import loadmodelingfunctions
import loaddata
import matplotlib.pyplot as plt
import seaborn as sns

%reload_ext autoreload
%autoreload 2

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": '16'
})

Daten laden

In [ ]:
stationen=loaddata.load_stations()
display(stationen)

In [ ]:
list_of_ss_groups=list()
list_of_ss_groups.insert(len(list_of_ss_groups),[3,4,7])                #SS-S
list_of_ss_groups.insert(len(list_of_ss_groups),[1,8])                  #SS-W1
list_of_ss_groups.insert(len(list_of_ss_groups),[5,6])                  #SS-W2
list_of_ss_groups.insert(len(list_of_ss_groups),[2])                    #SS-W3
list_of_ss_groups.insert(len(list_of_ss_groups),[0])                    #SS-W4

dict_group = loaddata.dict_groups()

n_clusters = len(list_of_ss_groups)
n_stations = 9
year = 2018

In [ ]:
list_power_d, list_power_yd, list_power_ysd, list_power_ymd, list_mat_power_d, list_power_ywd, list_power_yswd, list_power_only_d = loaddata.load_data_from_measurements([0,1,2,3,4,5,6,7,8], d=True, yd=True, ysd=True,ywd=True,yswd=True)

Gruppierte Daten bilden

In [ ]:
list_of_data_inputs = list()

for group in range(0, n_clusters):
    list_df_swd_group = list()
    this_group        = list_of_ss_groups[group]
    n_member          = len(this_group)

    count = 0
    for station in this_group:
        this_df_yswd = list_power_yswd[station][year]
        normed_yswd  = this_df_yswd/this_df_yswd.max().max()
        if count == 0:
            df_swd_group = normed_yswd.copy()
        else:
            df_swd_group = pd.concat([df_swd_group, normed_yswd], axis = 1, levels=['Jahreszeit'])
        count = count + 1
        
    count = 0
    for station in this_group:
        this_df_yd = list_power_yd[station][year]
        if count == 0:
            df_w_group = this_df_yd.copy()
            df_w_group = df_w_group/df_w_group.max().max()
        else:
            df_w_group = df_w_group + this_df_yd/this_df_yd.max().max()
        count = count + 1
    df_w_group = df_w_group/n_member
    
    this_data_input=[df_swd_group, df_w_group, df_swd_group]
    list_of_data_inputs.insert(len(list_of_data_inputs), this_data_input)

list_of_data_inputs_stations = list()
for station in range(0, n_stations):
    this_data_input=[list_power_d[station], list_power_yd[station][year], list_power_d[station]]
    list_of_data_inputs_stations.insert(len(list_of_data_inputs_stations), this_data_input)


Parametrisierung für Gruppen

In [ ]:
list_of_models_groups = list()                 #len = n_clusters

for group in range(0, n_clusters):
    group_model = StochasticLoadModel(normalization=[4,1,1,2],order_regression=6)
    group_model.parameterize(data_in = list_of_data_inputs[group], plots = False)
    list_of_models_groups.insert(len(list_of_models_groups), group_model)
    
list_of_models_stations = list()               #len = n_stations

for station in range(0, n_stations):
    station_model = StochasticLoadModel(normalization=[4,1,1,2],order_regression=6)
    station_model.parameterize(data_in = list_of_data_inputs_stations[station], plots = False)
    list_of_models_stations.insert(len(list_of_models_stations), station_model)

Simulation einzelner Ortsnetzstationen mit individueller Parametrisierung

In [ ]:
list_of_references_stations = list()                 #len = n_stations
list_of_results_stations    = list()                 #len = n_stations
list_of_mapes_stations      = list()                 #len = n_stations

n_runs = 1

MAPES=np.zeros([n_stations,3])

for station in range(0, n_stations):
    power_yd             = list_power_yd[station]
    reference_data       = power_yd[year]
    list_of_references_stations.insert(len(list_of_references_stations), reference_data)
    station_model        = list_of_models_stations[station]
    result_station       = station_model.simulate(n_runs = n_runs)
    energy_reference     = 0.25*reference_data.sum().sum()                                       #Denormalisierung nach Energie
    denormalized_result  = station_model.denormalize(result_station, new_case_4 = 4, energy = energy_reference)
    list_of_results_stations.insert(len(list_of_results_stations), denormalized_result)

    MAPEs, _, _, daily_MPE = denormalized_result.analyze(reference_data, plots=False, show_tables=False, export_plots=False)
    MAPES[station,:]       = MAPEs
    list_of_mapes_stations.insert(len(list_of_mapes_stations), daily_MPE)

Simulation einzelner Ortsnetzstationen mit Gruppen-Parametrisierung

In [ ]:
list_of_references_groups = list()                 #len = n_stations
list_of_results_groups    = list()                 #len = n_stations
list_of_mapes_groups      = list()                 #len = n_stations

n_runs = 1

MAPES=np.zeros([n_stations,3])

for station in range(0, n_stations):
    group_of_the_station = stationen.loc[:, 'Cluster'].iloc[station]
    index_of_the_group   = dict_group[group_of_the_station]
    power_yd             = list_power_yd[station]
    reference_data       = power_yd[year]
    list_of_references_groups.insert(len(list_of_references_groups), reference_data)
    station_model        = list_of_models_groups[index_of_the_group]
    result_station       = station_model.simulate(n_runs = n_runs)
    energy_reference     = 0.25*reference_data.sum().sum()                                       #Denormalisierung nach Energie
    denormalized_result  = station_model.denormalize(result_station, new_case_4 = 4, energy = energy_reference)
    list_of_results_groups.insert(len(list_of_results_groups), denormalized_result)

    MAPEs, _, _, daily_MPE = denormalized_result.analyze(reference_data, plots=False, show_tables=False, export_plots=False)
    MAPES[station,:]       = MAPEs
    list_of_mapes_groups.insert(len(list_of_mapes_groups), daily_MPE)